## 146. LRU Cache

In [25]:
class DoubleLinkedNode:
    def __init__(self, key=0, val=0, pre=None, nxt=None):
        self.key = key
        self.val = val
        self.pre = pre
        self.nxt = nxt

class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cur_capacity = 0
        self.memo = dict()
        self.head = DoubleLinkedNode()
        self.tail = DoubleLinkedNode()
        self.head.nxt = self.tail
        self.tail.pre = self.head

    def delete_node(self, node):
        node.pre.nxt = node.nxt
        node.nxt.pre = node.pre

    def add_node(self, node):
        node.nxt = self.head.nxt
        node.pre = self.head

        self.head.nxt.pre = node
        self.head.nxt = node

    def move_to_head(self, node):
        self.delete_node(node)
        self.add_node(node)


    def get(self, key: int) -> int:
        node = self.memo.get(key)
        if not node: return -1
        self.move_to_head(node)
        return node.val

    def put(self, key: int, value: int) -> None:
        node = self.memo.get(key)
        if node:
            node.val = value
            self.move_to_head(node)
        else:
            if self.cur_capacity == self.capacity:
                node = self.tail.pre
                if not node:
                    print(key, value)
                self.delete_node(node)
                del self.memo[node.key]
                self.cur_capacity -= 1
            node = DoubleLinkedNode(key, value)
            self.memo[key] = node
            self.add_node(node)
            self.cur_capacity += 1


In [26]:
lru = LRUCache(2)

lru.put(1, 1)
lru.put(2, 2)
lru.get(1)

insert 0 2 1 1
add (1, 1), cur capacity is 1
insert 1 2 2 2
add (2, 2), cur capacity is 2
here
get (1, 1), cur capacity is 2


1

We can also use the builtin OrderedDict

An OrderedDict is a dictionary subclass that remembers the order that keys were first inserted.
The only difference between dict() and OrderedDict() is that:

OrderedDict preserves the order in which the keys are inserted. A regular
 dict doesn’t track the insertion order, and iterating it gives the values in an
 arbitrary order. By contrast, the order the items are inserted is remembered by OrderedDict.



In [28]:
from collections import OrderedDict

In [34]:
od = OrderedDict()
od['a'] = 1
od['b'] = 2
od['c'] = 3

In [30]:
for key in od:
    print(key, od[key])

a 1
b 2
c 3


In [36]:
od.popitem(False)

('a', 1)

In [32]:
for key in od:
    print(key, od[key])

a 1
c 3


In [33]:
od.pop

<function OrderedDict.pop>

In [37]:
# the ordereddict version
class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.od = OrderedDict()

    def get(self, key: int) -> int:
        val = self.od.get(key)
        if not val: return -1
        self.od.move_to_end(key)
        return val

    def put(self, key: int, value: int) -> None:
        val = self.od.get(key)
        if val:
            self.od[key] = value
            self.od.move_to_end(key)
        else:
            if len(self.od) == self.capacity:
                # pop the first one.
                self.od.popitem(False)
            self.od[key] = value

In [39]:
lru = LRUCache(2)

lru.put(1, 1)
lru.put(2, 2)
lru.get(2)





2

## 单调栈问题
栈（stack）是很简单的一种数据结构，先进后出的逻辑顺序，符合某些问题的特点，比如说函数调用栈。
单调栈实际上就是栈，只是利用了一些巧妙的逻辑，使得每次新元素入栈后，栈内的元素都保持有序（单调递增或单调递减）。
听起来有点像堆（heap）？不是的，单调栈用途不太广泛，只处理一种典型的问题，叫做 Next Greater Element。本文用讲解单调队列的算法模版解决这类问题
，并且探讨处理「循环数组」的策略。

### 496. Next Greater Element I
You are given two arrays (without duplicates) nums1 and nums2 where nums1’s elements are
subset of nums2. Find all the next greater numbers for nums1's elements in the
corresponding places of nums2.

The Next Greater Number of a number x in nums1 is the first greater number to its
right in nums2. If it does not exist, output -1 for this number.

比如说，输入一个数组 nums = [2,1,2,4,3]，你返回数组 [4,2,4,-1,-1]。

解释：第一个 2 后面比 2 大的数是 4; 1 后面比 1 大的数是 2；第二个 2 后面比 2 大的数是 4; 4 后面没有比 4 大的数，填 -1；3 后面没有比 3 大的数，填 -1。
这道题的暴力解法很好想到，就是对每个元素后面都进行扫描，找到第一个更大的元素就行了。但是暴力解法的时间复杂度是 O(n^2)。
这个问题可以这样抽象思考：把数组的元素想象成并列站立的人，元素大小想象成人的身高。这些人面对你站成一列，如何求元素「2」的 Next Greater Number 呢？很简单，如果能够看到元素「2」，那么他后面可见的第一个人就是「2」的 Next Greater Number，因为比「2」小的元素身高不够，都被「2」挡住了，第一个露出来的就是答案。

In [1]:
class Solution:
    def nextGreaterElement(self, nums1: List[int], nums2: List[int]) -> List[int]:
        stack = []
        memo = dict()
        """
        保证stack里都是矮个子，遇到一个高个子，弹出一个矮个子。
        """
        for i in range(len(nums2)):
            while stack and nums2[i] > stack[-1]:
                memo[stack.pop()] = nums2[i]
            stack.append(nums2[i])
        while stack:
            memo[stack.pop()] = -1
        return [memo[n] for n in nums1]

NameError: name 'List' is not defined

### 739. Daily Temperature
Given a list of daily temperatures T, return a list such that, for each day in the input, tells you how many days you would have to wait until a warmer temperature. If there is no future day for which this is possible, put 0 instead.

For example, given the list of temperatures T = [73, 74, 75, 71, 69, 72, 76, 73], your output should be [1, 1, 4, 2, 1, 1, 0, 0].

Note: The length of temperatures will be in the range [1, 30000]. Each temperature will be an integer in the range [30, 100]

In [ ]:
class Solution:
    def dailyTemperatures(self, T: List[int]) -> List[int]:
        if not T: return []
        n = len(T)
        stack = []
        memo = [0] * n
        for i in range(n):
            while stack and T[stack[-1]] < T[i]:
                j = stack.pop()
                memo[j] = i - j
            stack.append(i)
        return memo


### Trie


In [ ]:
class Node:

    def __init__(self, val):
        self.val = val
        self.children = [None] * 26
        self.is_leaf = False

class Trie:

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.root = Node(-1)


    def insert(self, word: str) -> None:
        """
        Inserts a word into the trie.
        """
        cur = self.root
        for c in word:
            i = ord(c) - ord('a')
            if not cur.children[i]:
                cur.children[i] = Node(c)
            cur = cur.children[i]
        cur.is_leaf = True


    def search(self, word: str) -> bool:
        """
        Returns if the word is in the trie.
        """
        cur = self.root
        for c in word:
            i = ord(c) - ord('a')
            if not cur.children[i]:
                return False
            cur = cur.children[i]
        return cur.is_leaf

    def startsWith(self, prefix: str) -> bool:
        """
        Returns if there is any word in the trie that starts with the given prefix.
        """
        cur = self.root
        for c in prefix:
            i = ord(c) - ord('a')
            if not cur.children[i]:
                return False
            cur = cur.children[i]
        return True


# Your Trie object will be instantiated and called as such:
# obj = Trie()
# obj.insert(word)
# param_2 = obj.search(word)
# param_3 = obj.startsWith(prefix)